In [ ]:
# Libraries

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from bs4 import BeautifulSoup
import re
import tqdm
import unicodedata

from datetime import date
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# Random Forest model on BOW features
from sklearn.ensemble import BaggingClassifier

In [12]:
def data_preprocessing(path,method='training', cv_path):
        df=pd.read_csv(path,encoding = "ISO-8859-1")
        df['Buckets Items'].replace({"Environment Issue": "Application Issue","Timing Exception": "Application Issue", "Infrastrature Issue": "Data related Issue",
                     'Timing Issue' : 'Data related Issue', 'Latency related issue' : 'Application Issue' }, inplace=True)
        df['Predecessor TestCase ID'].fillna(0, inplace=True)
        df['Successor TestCase ID'].fillna(0, inplace=True)
        df['Execution Time'] = pd.to_datetime(df['Execution Time for test step'])
        df['exe_hour'] = df['Execution Time'].dt.hour
        df['exe_min'] = df['Execution Time'].dt.minute
        df['exe_sec'] = df['Execution Time'].dt.second
        df.drop(columns = {'Execution Time for test step','Execution Time'}, inplace=True)
        df['Exception Name'] =  df['Exception Name'].str.cat(df['RCA'],sep=" ")
        df['Exception Name'].fillna('', inplace=True)
        df.drop(columns={'RCA','Test Run Id'}, inplace=True)
        df['Exception Name'] = df['Exception Name'].astype('str')
        df1 = cv_conversion(pre_process_corpus(df['Exception Name']),method=method,cv_path)
        
        #Data_Preprocessing        
        #df1 = cv_conversion(pre_process_corpus(df[['Exception Name','RCA']]))
        
        if method == 'training':
            df['Buckets Items'].replace({"Environment Issue": "Application Issue","Timing Exception": "Application Issue", "Infrastrature Issue": "Data related Issue",
                     'Timing Issue' : 'Data related Issue', 'Latency related issue' : 'Application Issue' }, inplace=True)
            
            
            #df2 = cv_conversion(pre_process_corpus(df['RCA']))
           # df3 = pd.concat([df1,df2], axis=1)
            data = pd.concat([df1,df], axis=1)
            
            data['Buckets Items'] = data['Buckets Items'].replace({"Application Issue":0, "Automation Issue":1, "Data related Issue":2})
        else :
            data = pd.concat([df1,df], axis=1)
        
        data.drop(columns={'Exception Name'},inplace=True)
        
             
        #Target Renaming
        data = data.rename(columns={'Buckets Items' : 'FailureReason'})
        
        
        return data

SyntaxError: non-default argument follows default argument (<ipython-input-12-6899561d9a10>, line 1)

In [13]:

def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

In [14]:

#function defined to remove unicode data

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

In [15]:
# lower case and remove special characters\whitespaces

def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = strip_html_tags(doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = remove_accented_chars(doc)
    #doc = contractions.fix(doc)
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()
    doc=re.sub('[0-9]', '', doc)
    norm_docs.append(doc)
  
  return norm_docs

In [4]:
def cv_conversion(df,method):
    
    if method == 'training':
        #df['Exception Name'] =  df['Exception Name'].str.cat(df['RCA'],sep=" ")
        #df['Exception Name'].fillna('', inplace=True)
        #print (combined_2)
        cv = CountVectorizer(ngram_range=(1, 1), min_df=0.001, max_df=0.99)
        cv_exception_matrix = cv.fit_transform(df)
        pickle.dump(cv, open(path, 'wb'))
    else:
        cv = pickle.load(open(path,'rb'))
        cv_exception_matrix = cv.transform(df)
        
    
    cv_exception_matrix=cv_exception_matrix.toarray()
    # get all unique words in the corpus
    vocab = cv.get_feature_names()
    # show document feature vectors
    data2=pd.DataFrame(cv_exception_matrix, columns=vocab)
        
    return data2

In [5]:
def model_training(data, path):
    x = data.loc[:, data.columns != 'FailureReason']
    y = data['FailureReason']
    model = BaggingClassifier(n_estimators=135)
    
    model = model.fit(x,y)
        
#     path = "\".join([path, filename])
    pickle.dump(model, open(path, 'wb'))
    
    return 'Model has been trained successfully'

In [6]:
def model_predict(data,path):
    x = data.loc[:, data.columns != 'FailureReason']    
    # load the model from disk
    loaded_model = pickle.load(open(path,'rb'))
    
    result = loaded_model.predict(x)
    pred_result = pd.DataFrame(result, columns=['FailureReason'])
    pred_result['FailureReason'] = pred_result['FailureReason'].replace({0:'Application Issue',1: 'Automation Issue', 2: 'Data related Issue'})
    #data = pd.concat([x,pred_result], axis=1)
#     filename = 'output_{}.csv'.format(date.today())
#     data.to_csv(filename,index=False)
    
    return pred_result  
    

In [7]:
def output(data,path='UC_Sample Data.csv'):
    ip = pd.read_csv(path,encoding = "ISO-8859-1")
    ip.drop(columns = 'Buckets Items', inplace=True)
    ip = ip.rename(columns={'Test Case ID' : 'TestCaseID',
                            'Failed Steps' : 'FailedSteps',
                            'Exception Name': 'ExceptionName'})
    op = pd.concat([ip,data], axis=1)
    filename = 'output_{}.csv'.format(date.today())
    op.to_csv(filename,index=False)
    df = pd.DataFrame(op, columns = ['TestCaseID','FailedSteps','ExceptionName','FailureReason'])
    output  = df.to_json(r'UC_27.json',orient='records')
    
    return output

In [8]:
import os

os.getcwd()

'C:\\Users\\45448\\Documents\\ATOP\\UC_27'

In [9]:
def main(data, method,path = 'C:\\Users\\45448\\Documents\\ATOP\\UC_27\\uc27_train',
         cvpath='C:\\Users\\45448\\Documents\\ATOP\\UC_27\\'):
    if method == 'training' :
        df_train = data_preprocessing(path = 'UC_Sample Data.csv',method=method)
        status = model_training(df_train, path)
    
    else :
        df_pred = data_preprocessing(path = 'UC_Sample Data.csv',method=method)
        df_pred = model_predict(df_pred,'C:\\Users\\45448\\Documents\\ATOP\\UC_27\\uc27')
        print ('Model has predicted the Failure Reason')
        op = output(df_pred)
    return op if status is None else status

In [ ]:
df = data_preprocessing()
df
#model_training(df, 'C:\\Users\\45448\\Documents\\ATOP\\UC_27\\')

In [10]:
df_train = data_preprocessing(path = 'UC_Sample Data.csv',method='training')

NameError: name 'data_preprocessing' is not defined

In [133]:
df_train.isna().sum()

acceptinsecurecerts      0
acceptsslcerts           0
access                   0
accessible               0
accessing                0
                        ..
Successor TestCase ID    0
FailureReason            0
exe_hour                 0
exe_min                  0
exe_sec                  0
Length: 400, dtype: int64

In [134]:
model_training(df_train, os.getcwd())

Model has been successfully trained


In [148]:
os.path.dirname(os.path.realpath(__file__))

NameError: name '__file__' is not defined

In [144]:
df_pred = data_preprocessing(path = 'UC_Sample Data.csv',method='predict')

100%|██████████| 250/250 [00:00<00:00, 4222.54it/s]


In [145]:
df_pred = model_predict(df_pred,'C:\\Users\\45448\\Documents\\ATOP\\UC_27\\uc27_2020-08-17' )

In [146]:
output(df_pred)